In [10]:
!pip install fast_autocomplete
!pip install pylev

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


# AUTOCOMPLETE BY OBJECT METHOD

In [4]:
import re
from collections import Counter
import json
import pandas as pd
from collections import Iterable
import collections
# import boto3
from io import BytesIO,StringIO
from fast_autocomplete import AutoComplete


def clean(text):
    text=str(text)
    text=re.sub(r"[`:,/!$%^&*|+'\\]","",text.lower())
    text=text.strip().replace("  "," ")
    return text
def makeQuery(texts,indexes):
    for text,indx in zip(texts,indexes):
        words=text.split()
        for i,w in enumerate(words):
            yield " ".join(words[i:])+" "+str(indx)


def fit(df):
    texts=df['Text'].values()
    indx=[]
    bkg_queries=[]
    for i,txt in enumerate(texts):
        bkg_queries.append(clean(txt))
        indx.append(i)
    bkg_query={txt:{} for txt in makeQuery(bkg_queries,indx)}
    autocomplete = AutoComplete(words=bkg_query)
    return bkg_queries,autocomplete

def find(text,inputDict):
    score={'Name':1,"SurName":1,"qulification":5,"Mobile":5,"Email":5,"Work":5,"Location":2,"company":2}
    scored=0
    main_texted=[mysplit(s) for s in text.split()]       
    main =list(flatten(main_texted))
    for key in inputDict.keys():
        for v in inputDict[key]:
            test_list = [i for i in main if i]        
            word = str(v)
            word = word.lower()
            occurrences = collections.Counter(test_list)
            n = occurrences[word]
            scored += n * score[key]
    return scored
    
def flatten(lis):
     for item in lis:
            if isinstance(item, Iterable) and not isinstance(item, str):
                for x in flatten(item):
                    yield x
            else:        
                 yield item
def mysplit(s):
    main_lsit= []
    head = s.rstrip('0123456789')
    tail = s[len(head):]
    
    main_lsit.append(head)
    main_lsit.append(tail)
    return main_lsit

def predict(inputDict,paragraph,candidate_generator):
    indexes=[]
    for key in inputDict.keys():
        for v in inputDict[key]:
            if str(v).startswith("+91"):
                v=str(v)[3:]
            results=candidate_generator.search(str(v).lower(), max_cost=0, size=50)
            indexes.extend([ int(r[0].split()[-1]) for r in results])
    output_data = {"index":[],"text":[],"score":[],"input":[]}
    for index in set(indexes):
        text=paragraph[index]
        score=find(text,inputDict)
        output_data['index'].append(index)
        output_data['text'].append(text)
        output_data['score'].append(score)
    return json.dumps(output_data)
  

def lambda_handler(event,context):
    paragraph,modelgen=fit(event)
    res=predict(event['input']['0'],paragraph,modelgen)
    return res


In [2]:
data={
  "Text": {
    "0": "technopolis plans to develop in stages an area of no less than 100000 sourabh square meters in order to host companies working in computer technologies and telecommunications the statement said .",
    "1": "the international electronic industry company elcoteq has laid off tens of employees from its tallinn facility ; contrary to earlier layoffs the company contracted the 9530450649 ranks of its office workers the daily postimees reported .",
    "2": "with the new production plant the company would increase its capacity to meet the expected increase shivam in demand and would improve the use of raw materials and therefore increase the production profitability .",
    "3": "according to the company s updated strategy for the years 2009-2012 basware targets a long-term net sales growth in the sharma range of 20 -40 with an operating profit margin of 10 -20 of net sales .",
    "4": "financing of aspocomp s growth aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding hdi printed circuit boards pcbs ."
  },
  "input": {
    "0": {
      "Name": [
        "sourabh",
        "shivam"
      ],
      "SurName": [
        "Aggarwal"
      ],
      "qulification": [
        "Blackhat",
        "b.tech"
      ],
      "Mobile": [
        9530450649,
        7014741261
      ],
      "Email": [
        "100rabh1210@gmail.com",
        "sourabh12sharma1@gmail.com"
      ],
      "Work": [
        "act",
        "PIEOBJECT"
      ]
    }
  }
}

In [5]:
lambda_handler(data)

'{"index": [0, 1, 2, 3], "text": ["technopolis plans to develop in stages an area of no less than 100000 sourabh square meters in order to host companies working in computer technologies and telecommunications the statement said .", "the international electronic industry company elcoteq has laid off tens of employees from its tallinn facility ; contrary to earlier layoffs the company contracted the 9530450649 ranks of its office workers the daily postimees reported .", "with the new production plant the company would increase its capacity to meet the expected increase shivam in demand and would improve the use of raw materials and therefore increase the production profitability .", "according to the company s updated strategy for the years 2009-2012 basware targets a long-term net sales growth in the sharma range of 20 -40 with an operating profit margin of 10 -20 of net sales ."], "score": [1, 5, 1, 0], "input": []}'

In [6]:
import re
from collections import Counter
import json
import pandas as pd
from fast_autocomplete import autocomplete_factory
from collections import Iterable
import collections
import re
from sagemaker import get_execution_role
import boto3
from io import BytesIO,StringIO
import json,time


def clean(text):
    text=str(text)
    text=re.sub(r"[`:,/!$%^&*|+'\\]","",text.lower())
    text=text.strip().replace("  "," ")
    return text
def makeQuery(texts,indexes):
    for text,indx in zip(texts,indexes):
        words=text.split()
        for i,w in enumerate(words):
            yield " ".join(words[i:])+" "+str(indx)


def fit(df):
    if type(df)==dict:
        df=pd.DataFrame(df)
    df['Text']=df['Text'].apply(func=clean)
    paragraph = df['Text'].values
    bkg_queries=df.Text
    bkg_query_count = Counter(list(makeQuery(bkg_queries,bkg_queries.index)))
    query_log = dict()
    for query, count in bkg_query_count.items():
        query_log[query] = [dict(), query, count]
    with open('query_log.json', 'w') as outfile:
        json.dump(query_log, outfile, indent=2)
    return paragraph

def find(text,inputDict):
    score={'Name':1,"SurName":1,"qulification":5,"Mobile":5,"Email":5,"Work":5,"Location":2,"company":2}
    scored=0
    main_texted=[mysplit(s) for s in text.split()]       
    main =list(flatten(main_texted))
    for key in inputDict.keys():
        for v in inputDict[key]:
            test_list = [i for i in main if i]        
            word = str(v)
            word = word.lower()
            occurrences = collections.Counter(test_list)
            n = occurrences[word]
            scored += n * score[key]
    return scored
    
def flatten(lis):
     for item in lis:
            if isinstance(item, Iterable) and not isinstance(item, str):
                for x in flatten(item):
                    yield x
            else:        
                 yield item
def mysplit(s):
    main_lsit= []
    head = s.rstrip('0123456789')
    tail = s[len(head):]
    
    main_lsit.append(head)
    main_lsit.append(tail)
    return main_lsit

def predict(inputDict,paragraph,candidate_generator):
    indexes=[]
    for key in inputDict.keys():
        for v in inputDict[key]:
            if str(v).startswith("+91"):
                v=str(v)[3:]
            results=candidate_generator.search(str(v).lower(), max_cost=0, size=50)
            indexes.extend([ int(r[0].split()[-1]) for r in results])
    output_data = pd.DataFrame(columns=["index","text","score"])
    count=0
    for index in set(indexes):
        print()
        text=paragraph[index]
        score=find(text,inputDict)
        output_data.loc[count]=([index,text,score])
        count+=1
    return output_data.sort_values('score',axis=0,ascending=False)

def model():
    content_files = {
        'words': {
            'filepath': 'query_log.json',
            'compress': True  # means compress the graph data in memory
        }
    }
    return autocomplete_factory(content_files=content_files)

def execute(event):
    global data
    bucket = 'uploadjsonfolder'
    client = boto3.client('s3')
    s3 = boto3.resource('s3')
    paragraph=fit(event)
    modelgen=model()
    res=predict(event['input']['0'],paragraph,modelgen)
    return res




In [7]:
events = {"Text":{"0":"TeliaSonera sourabh12sharma1@gmail.com TLSN said the offer is in line with its strategy to increase its ownership in sourabh sharma is b.tech core business holdings and would strengthen Eesti Telekom 's offering to its customers .","1":"( Filippova ) A 7727805159 trilateral sourabh12sharma1@gmail.com agreement on investment in the construction of a technology park in St Petersburg sourabh sharma is 7727805159 was to have been signed in the course of the forum , Days of the Russian Economy , that opened in Helsinki today .","2":"STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four sourabh123 of the largest Nordic forestry groups .","3":"Lifetree was founded in 2000 , and its revenues have risen on an average by 40 % with margins in late 30s ."},
          "input":{"0":{"Name":["sourabh","shivam"],"SurName":["Aggarwal"],"qulification":["Blackhat","b.tech"],"Mobile":[7727805159,7014741261],"Email":["100rabh1210@gmail.com","sourabh12sharma1@gmail.com"],"Work":["act","PIEOBJECT"]},"1":None,"2":None,"3":None}}

In [8]:
start=time.time()
res=execute(events)
print("Time took to execute: ",time.time()-start)





Time took to execute:  0.11513519287109375


In [9]:
res

,index,text,score
1,1,( filippova ) a 7727805159 trilateral sourabh1...,16
0,0,teliasonera sourabh12sharma1@gmail.com tlsn sa...,11
2,2,stora enso norske skog m-real upm-kymmene cred...,1
3,3,lifetree was founded in 2000 and its revenues ...,0
